In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minzzzzi","key":"74a980fb52033cf0ead89c84120a4a9c"}'}

In [2]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle

Uninstalling kaggle-1.5.10:
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 1.5MB 12.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 59 kB 4.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-py3-none-any.whl size=73268 sha256=334da50af29ef7ef91c126f10519d5758c9fd123a77a327cd46ef2ee2ae131a3
  Stored in directory: /root/.cache/pip/wheels/1c/dd/dd/c493e6f981182c1411e288c553310f76e212bac3afbdac1294
Successfully built kaggle


In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c childpark

  0% 0.00/41.2k [00:00<?, ?B/s]
100% 41.2k/41.2k [00:00<00:00, 36.9MB/s]


In [5]:
!unzip childpark.zip

Archive:  childpark.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train_classi.csv        


In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [3]:
train = pd.read_csv('train_classi.csv',header=None,skiprows=1)
test = pd.read_csv('test.csv',header=None,skiprows=1)
submit = pd.read_csv('submit.csv')

#train[:5]

In [13]:
submit[:5]

,id,result
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN


In [4]:
x_train = train.loc[:,1:8]
y_train = train.loc[:,9]

x_test = test.loc[:,1:8]

In [5]:
# Data Load

date = train.loc[:,0]
A = date.str.extract(r'(\d+)[:]', expand=True)
x_train["data1"] = A.astype(int)
x_train.apply(pd.to_numeric)


date = test.loc[:,0]
A = date.str.extract(r'(\d+)[:]', expand=True)
x_test["data1"] = A.astype(int)
x_test.apply(pd.to_numeric)

,1,2,3,4,5,6,7,8,data1
0,11.5,0.0,0.7,67,0,3,0,128,5
1,11.3,0.0,0.3,69,0,7,0,199,6
2,11.5,0.0,0.8,68,0,3,0,343,7
3,12.9,0.0,1.0,59,0,0,0,678,8
4,14.6,0.0,1.7,57,0,0,0,618,9
...,...,...,...,...,...,...,...,...,...
615,18.2,0.0,4.0,53,0,0,0,794,20
616,17.3,0.0,2.4,56,0,0,0,816,21
617,16.8,0.0,2.6,56,0,0,0,882,22
618,16.5,0.0,2.2,58,0,0,0,623,23


In [6]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()

x_train_std = std.fit_transform(x_train)
x_test_std = std.fit_transform(x_test)

---
## 차원 축소 + 표준화된 데이터 + SVM
---

In [21]:
#LDA 로 차원 축소 후 SVM을 이용

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components = 2)
x_train_lda = lda.fit_transform(x_train_std,y_train)
x_test_lda = lda.transform(x_test_std)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {'C' : [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma':[0.0001,0.0005,0.001,0.01,0.1]}

clf = GridSearchCV(SVC(class_weight = 'balanced'),param_grid,cv = 5)
clf.fit(x_train_lda, y_train)

In [23]:
from sklearn.metrics import accuracy_score

train_pred = clf.predict(x_train_lda)
print(accuracy_score(train_pred,y_train))

0.766588862954318


In [24]:
predict = clf.predict(x_test_lda)

for i in range(len(predict)):
  submit['result'][i] = predict[i].astype(int)

submit.to_csv('submission.csv',header=True,index=None)
!kaggle competitions submit -c childpark -f submission.csv -m "clf"

100% 3.54k/3.54k [00:05<00:00, 709B/s]
Successfully submitted to SejongAI.텀프로젝트.[어린이대공원역 시간대별 사용인원 예측]

---
## 차원축소 X 표준화 데이터 + SVM
---
- 너무 오래걸려서 X

In [ ]:
#차원 축소 안하고 SVM 이용
param_grid = {'kernel':('linear', 'rbf'),
              'C' : [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma':[0.0001,0.0005,0.001,0.01,0.1]}

clf = GridSearchCV(SVC(class_weight = 'balanced'),param_grid,cv = 5)
clf.fit(x_train_std, y_train)

train_pred = clf.predict(x_train_std)
print(accuracy_score(train_pred,y_train))

---
## 앙상블 모델
---

In [7]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=1,
                              criterion='gini',
                              class_weight = 'balanced')

In [11]:
# 앙상블 모델을 이용해 분류
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier

# 1 아다부스트
ada = AdaBoostClassifier(base_estimator = tree,
                         n_estimators=500,
                         learning_rate = 0.1,
                         random_state=1)

ada.fit(x_train_std,y_train)
train_pred = ada.predict(x_train_std)
print(accuracy_score(train_pred,y_train))

0.6672224074691564


In [20]:
predict = ada.predict(x_test_std)

for i in range(len(predict)):
  submit['result'][i] = predict[i].astype(int)

submit.to_csv('submission.csv',header=True,index=None)
!kaggle competitions submit -c childpark -f submission.csv -m "ada"

100% 3.54k/3.54k [00:06<00:00, 591B/s]
Successfully submitted to SejongAI.텀프로젝트.[어린이대공원역 시간대별 사용인원 예측]

In [12]:
#엑스트라 트리

dtc = ExtraTreesClassifier(n_estimators=200,criterion='gini',class_weight='balanced')

dtc.fit(x_train_std,y_train)
train_pred = dtc.predict(x_train_std)

print(accuracy_score(train_pred,y_train))

1.0


In [19]:
predict = dtc.predict(x_test_std)

for i in range(len(predict)):
  submit['result'][i] = predict[i].astype(int)

submit.to_csv('submission.csv',header=True,index=None)
!kaggle competitions submit -c childpark -f submission.csv -m "extra dtc"

100% 3.54k/3.54k [00:04<00:00, 755B/s]
Successfully submitted to SejongAI.텀프로젝트.[어린이대공원역 시간대별 사용인원 예측]